In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd
!pip install tensorflow

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.4.0'

# Evaluation

This database was created to identify horse race results, based upon the following variables: dressage, stad jump and time, cross-country jump and time, final score and final place. 

## Results
The following results were evaluated:

*E – Eliminated by the rules
*TE – Technical Elimination.
*RF – Rider Fall.
*W – rider chose to Withdraw before beginning the
phase.
*R – Rider chose to Retire during the phase.
*MR – Mandatory Retirement.
*DQ – Disqualified.
*DR – Dangerous Riding.
*NA – Not Accepted by the Ground Jury.

In [4]:
voice = pd.read_csv('../Resources/test_AT.csv')
voice.head()

,Dress Score,Stad Jump,Stad Time,XC Jump,XC Time,Final Score,Final Place,Result
0,37.0,0,0.0,0,0.0,37.0,1,CLASSIFIED
1,34.3,4,0.0,0,0.0,38.3,2,CLASSIFIED
2,37.8,0,7.6,0,0.0,45.4,3,CLASSIFIED
3,35.8,0,0.0,0,10.4,46.2,4,CLASSIFIED
4,34.0,0,0.0,0,20.0,54.0,5,CLASSIFIED


## Data Pre-Processing

In [5]:
X = voice.drop("Result", axis=1)
y = voice["Result"]
print(X.shape, y.shape)

(53214, 7) (53214,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=7))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=9, activation='softmax'))

In [13]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 909       
Total params: 11,809
Trainable params: 11,809
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
1248/1248 - 1s - loss: 0.2783 - accuracy: 0.9166
Epoch 2/60
1248/1248 - 1s - loss: 0.1634 - accuracy: 0.9335
Epoch 3/60
1248/1248 - 1s - loss: 0.1591 - accuracy: 0.9350
Epoch 4/60
1248/1248 - 1s - loss: 0.1568 - accuracy: 0.9350
Epoch 5/60
1248/1248 - 1s - loss: 0.1549 - accuracy: 0.9357
Epoch 6/60
1248/1248 - 1s - loss: 0.1531 - accuracy: 0.9366
Epoch 7/60
1248/1248 - 1s - loss: 0.1520 - accuracy: 0.9363
Epoch 8/60
1248/1248 - 1s - loss: 0.1513 - accuracy: 0.9374
Epoch 9/60
1248/1248 - 1s - loss: 0.1510 - accuracy: 0.9365
Epoch 10/60
1248/1248 - 1s - loss: 0.1505 - accuracy: 0.9374
Epoch 11/60
1248/1248 - 1s - loss: 0.1499 - accuracy: 0.9373
Epoch 12/60
1248/1248 - 1s - loss: 0.1494 - accuracy: 0.9377
Epoch 13/60
1248/1248 - 1s - loss: 0.1491 - accuracy: 0.9373
Epoch 14/60
1248/1248 - 1s - loss: 0.1488 - accuracy: 0.9375
Epoch 15/60
1248/1248 - 1s - loss: 0.1484 - accuracy: 0.9382
Epoch 16/60
1248/1248 - 1s - loss: 0.1477 - accuracy: 0.9377
Epoch 17/60
1248/1248 - 1s - loss

## Quantify our Trained Model

In [16]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

416/416 - 0s - loss: 0.1284 - accuracy: 0.9505
Normal Neural Network - Loss: 0.1284453123807907, Accuracy: 0.9505411982536316


## Make Predictions

In [17]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

C:\Users\atrav\Anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [18]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['CLASSIFIED' 'CLASSIFIED' 'CLASSIFIED' 'CLASSIFIED' 'CLASSIFIED']
Actual Labels: ['CLASSIFIED', 'CLASSIFIED', 'CLASSIFIED', 'CLASSIFIED', 'CLASSIFIED']
